In [30]:
import pandas as pd
import numpy as np
import os
from itertools import combinations
import math

#VARIABLES

NUMBER_OF_SLOTS = 3



In [31]:
# Manually read in data file

file_path = os.path.join('data', 'test_data.csv')

# Read the file into a pandas DataFrame

df = pd.read_csv(file_path)

# Drop subject column
df = df.drop(columns="Subject")   
print(df.head())

   Biology  Maths  Chemistry  Physics  Economics
0        1      1          0        0          0
1        1      1          0        0          0
2        1      1          0        0          0
3        1      1          0        0          0
4        1      1          0        0          0


In [32]:
# Number of subjects
number_of_subs = len(df.columns)
# Create subject list 1

subject_1_list = []
for n in range(0, number_of_subs-1):
    for i in range (1, number_of_subs-n):
        subject_1_list.append(df.columns[n])
print(subject_1_list)

['Biology', 'Biology', 'Biology', 'Biology', 'Maths', 'Maths', 'Maths', 'Chemistry', 'Chemistry', 'Physics']


In [33]:
# Create subject list 2
subject_2_list = []
for n in range(0, number_of_subs-1):
    for i in range (1, number_of_subs-n):
        subject_2_list.append(df.columns[i+n])
print(subject_2_list)

if len(subject_1_list) == len(subject_2_list):
    print("List are the same length")
else:
    print("Lists are different lengths, PLEASE CHECK")

['Maths', 'Chemistry', 'Physics', 'Economics', 'Chemistry', 'Physics', 'Economics', 'Physics', 'Economics', 'Economics']
List are the smame length


In [34]:
# Build clashes data

Clashes = []

for i in range(0, len(subject_1_list)):
    Count = 0
    for n in range(0, len(df)):
        if df[subject_1_list[i]].iloc[n] == 1 and df[subject_2_list[i]].iloc[n]:
            Count += 1
    Clashes.append(Count)
    
print(Clashes)
        

[5, 2, 4, 1, 3, 2, 5, 1, 3, 3]


In [35]:
# We list all pairs of subjects twice (in both orders)
df = pd.DataFrame({
    "Subject_1": subject_1_list + subject_2_list,
    "Subject_2": subject_2_list + subject_1_list,
    "Clashes": Clashes + Clashes
})

In [36]:
#TO DO

# 1. Make number of slots a variable
# 2. Look at mapping the optimal slots onto a timetable
# 3. Test google form data template and edit data ingress to check - 
#    if we could build and do this we can see if people in the room https://docs.google.com/forms/d/1eou9g1pQeVUu2bG3oOVr-EcGuSQQ953Vcb3-fpBC9N4/edit
# 4. Could you add extra constraints e.g. if all people studying x also study y (even if x is small) don't group them together because it 
# essentially creates another exam slot from the double booking

In [37]:
#Edit variable at top to change the number of slots
no_of_slots = NUMBER_OF_SLOTS

slot_dict = {}
for i in range(1, no_of_slots+1):  # Create 5 lists
    list_name = f"slot_{i}"
    slot_dict[list_name] = []
    
print(slot_dict)

{'slot_1': [], 'slot_2': [], 'slot_3': []}


In [38]:
# These are the variables where we will store the minimum number of clashes achieved
# And the subject combinations that achieve them
min_score = float('inf')

#Original code
no_of_slots = 2
slot_1 = []
slot_2 = []

# We list the distinct subjects
subjects = list(set(subject_1_list + subject_2_list))
num_subjects = len(subjects)

# We loop over all possible numbers of subjects to have in the first slot
for i in range(math.ceil(num_subjects/no_of_slots), num_subjects + 1):
    # Generate all combinations of subjects of size i
    subject_subsets = list(combinations(subjects, i))
    
    # We loop over every set of subjects of a particular size
    for sub_1 in subject_subsets:
        sub_1 = list(sub_1)
        sub_1_pairs = list(combinations(sub_1, 2))
        sub_1_score = 0
        
        # We loop over every pair of subjects within the subset to find the total number of clashes
        for pair in sub_1_pairs:
            pair_clashes = df.loc[(df['Subject_1'] == pair[0]) & (df['Subject_2'] == pair[1]), 'Clashes']
            if not pair_clashes.empty:
                sub_1_score += pair_clashes.iloc[0]
        
        # We work out the number of clashes in the second slot
        sub_2 = [s for s in subjects if s not in sub_1]
        sub_2_score = 0
        
        if len(sub_2) >= 2:
            sub_2_pairs = list(combinations(sub_2, 2))
            for pair in sub_2_pairs:
                pair_clashes = df.loc[(df['Subject_1'] == pair[0]) & (df['Subject_2'] == pair[1]), 'Clashes']
                if not pair_clashes.empty:
                    sub_2_score += pair_clashes.iloc[0]
        
        score = sub_1_score + sub_2_score
        
        # If the number of clashes with this timetabling arrangement is less than the previous minimum
        # We update the minimum score and the subjects that go in the two slots
        if score < min_score:
            min_score = score
            slot_1 = sub_1
            slot_2 = sub_2

print(f"The minimum number of clashes is {min_score} achieved by having the two slots be {slot_1} and {slot_2}")

The minimum number of clashes is 7 achieved by having the two slots be ['Chemistry', 'Maths', 'Physics'] and ['Biology', 'Economics']
